In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My\ Drive/ImageClassification

/content/drive/My Drive/ImageClassification


In [ ]:
import torch

print('CUDA GPU availalbe : {}'.format(torch.cuda.is_available()))
try:
    print('{} GPU(s) is(are) allocated'.format(torch.cuda.device_count()))
except:
    print('GPUs are not allocated. Current runtime is on CPU.')

CUDA GPU availalbe : True
1 GPU(s) is(are) allocated


In [ ]:
!tar Cxfz . data_2500.tar.gz

In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-cp36-none-any.whl size=16031 sha256=441537c3a3bf19f6dac678fe03049609cb6e03872ff6cf341d31e8240575a716
  Stored in directory: /root/.cache/pip/wheels/e9/c6/e1/7a808b26406239712cfce4b5ceeb67d9513ae32aa4b31445c6
Successfully built efficientnet-pytorch


In [ ]:
from glob import glob

ti = glob('./data/train/images/*')
ta = glob('./data/train/annotations/*')
vi = glob('./data/val/images/*')
va = glob('./data/val/annotations/*')

if len(ti) == len(ta):
    print('train data ({} files) unzip succeed'.format(len(ti)))
else:
    print('train data unzip failed')
  
if len(vi) == len(va):
    print('val data ({} files) unzip succeed'.format(len(vi)))
else:
    print('val data unzip failed')

train data (2500 files) unzip succeed
val data (500 files) unzip succeed


In [ ]:
import torch
from collections import deque, defaultdict
import os
from datetime import timedelta
import time


def label2logit(label):
    label_dict = {
        '타워' : 0,
        '다리' : 1,
        '빌딩' : 2,
        '센터' : 3,
        '전망대' : 4
    }
    
    return label_dict[label] 


class AverageMeter:
    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.
        self.count = 0
        self.fmt = fmt
    
    def update(self, val, n=1):
        self.deque.append(val)
        self.count += n
        self.total += val * n
        
    @property
    def median(self):
        data = torch.tensor(list(self.deque))
        return data.median().item()
    
    @property
    def avg(self):
        data = torch.tensor(list(self.deque), dtype=torch.float32)
        return data.mean().item()
    
    @property
    def global_avg(self):
        return self.total / self.count
    
    @property
    def max(self):
        return max(self.deque)
    
    @property
    def value(self):
        return self.deque[-1]
    
    def __str__(self):
        return self.fmt.format(
            median=self.median,
            max=self.max,
            avg=self.avg,
            global_avg=self.global_avg,
            value=self.value
        )
        
    
class MetricLogger:
    def __init__(self, delimeter='\t'):
        self.meters = defaultdict(AverageMeter)
        self.delimeter = delimeter
    
    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)
    
    def __getattr__(self, attr):
        if attr in  self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError("'{}' object does not have attribute '{}'".format(type(self).__name__, attr))
    
    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append("{}: {}".format(name, str(meter)))
        return self.delimeter.join(loss_str)
    
    def add_meter(self, name, meter):
        self.meters[name] = meter
    
    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ''
        start = time.time()
        end = time.time()
        iter_time = AverageMeter(fmt="{avg:.4f}")
        space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
        
        log_msg =  self.delimeter.join([
            header, 
            '[{0' + space_fmt + '}/{1}]',
            'eta: {eta}',
            '{meters}'
        ])
        
        for obj in iterable:
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(timedelta(seconds=int(eta_seconds)))
                print(log_msg.format(i, len(iterable), eta=eta_string, meters=str(self)))
            i += 1
            end = time.time()
            

def save_model(model, optimizer, epoch, step, cfg, is_final=False):
    if not os.path.exists(cfg.BASE_SAVE_DIR):
        os.mkdir(cfg.BASE_SAVE_DIR)
        
    if not os.path.exists(cfg.SAVE_DIR):
        os.mkdir(cfg.SAVE_DIR)
    
    if is_final:
        step = 'final'
    file_name = '{}_{}_{}.pth'.format(cfg.MODEL.ARCH, cfg.TEAM_NAME, step)
    file_path = os.path.join(cfg.SAVE_DIR, file_name)
    torch.save({
            "model" : model.state_dict(),
            "opt" : optimizer.state_dict(),
            "step" : step, 
            "epoch" : epoch, 
        }, file_path)
    
    return file_path


class AttributeDict(dict):
    def __init__(self):
        self.__dict__ = self
        

class ConfigTree:
    def __init__(self):
        self.DATASET = AttributeDict()
        self.SYSTEM = AttributeDict()
        self.TRAIN = AttributeDict()
        self.MODEL = AttributeDict()

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
from PIL import Image
import json
from glob import glob
import os
from pathlib import Path


class LandMarkDataset(Dataset):
    def __init__(self, root, cfg, mode='train', transform=None):
        """
        root : './data/'
        """
        super(LandMarkDataset, self).__init__()
        self.root = os.path.join(root, mode)
        self.cfg = cfg
        self.transform = transform
        self.images = sorted(glob(self.root + '/images/*.jpg'))
        self.annotations = sorted(glob(self.root + '/annotations/*.json'))
        
    def __getitem__(self, idx):
        image = self.images[idx]
        image = Image.open(image)
        img_w, img_h = image.size
        
        annotation = self.annotations[idx]
        label = self.get_gt(annotation)
        label =  label2logit(label)
        if self.transform:
            image = self.transform(image)
        
        return image, label
        
    def __len__(self):
        assert len(self.images) == len(self.annotations), "# of image files and # of json files do not match"
        return len(self.images)
    
    def get_gt(self, json_file):
        json_file = Path(json_file)
        with open(json_file, 'r') as f:
            annotation = json.load(f)
        
        gt_class = annotation['regions'][0]['class'] 
        
        return gt_class


def get_loader(dataset, cfg, shuffle=True):
    return DataLoader(dataset, batch_size=cfg.TRAIN.BATCH_SIZE, shuffle=shuffle)


def get_augmentation(size=224, use_flip=True, use_gray_scale=False, use_normalize=False, use_advprop=False):
    resize_crop = transforms.RandomResizedCrop(size=size)
    random_flip = transforms.RandomHorizontalFlip(p=0.5)
    gray_scale = transforms.RandomGrayscale(p=0.2)
    normalize = transforms.Lambda(lambda img: img*2.0 - 1.0) if use_advprop else transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    to_tensor = transforms.ToTensor()
    
    transforms_array = np.array([resize_crop, random_flip, gray_scale, to_tensor, normalize])
    transforms_mask = np.array([True, use_flip, use_gray_scale, True, use_normalize])
    
    transform = transforms.Compose(transforms_array[transforms_mask])
    
    return transform

In [ ]:
import torchvision 
from torchvision import models
import torch.nn as nn
from efficientnet_pytorch import EfficientNet


def get_convnet(model_arch='vgg11', use_pretrained=False, num_classes=5, use_advprop=False):
    """
    model_arch : torchvision.models에 미리 정의 되어 있는 model을 지정
        예 : 'vgg11', 'vgg11_bn', 'vgg_13', 'vgg_13_bn'
    """
    if model_arch.startswith('efficientnet'):
        model = EfficientNet.from_pretrained(model_arch, advprop=use_advprop)
        in_feat = model._fc.in_features
        model._fc = nn.Linear(in_feat, num_classes)
        return model
    elif use_pretrained:
        model = models.__dict__[model_arch](pretrained=True)
        if model_arch.startswith('vgg') or model_arch.startswith('alexnet'):
            in_features = model.classifier[6].in_features
            model.classifier[6] = nn.Linear(in_features, num_classes)
        elif model_arch.startswith('resnet'):
            in_features = model.fc.in_features
            model.fc = nn.Linear(in_features, num_classes)
        elif model_arch.startswith('squeezenet'):
            model.num_classes = num_classes
            in_channels = model.classifier[1].in_channels
            model.classifier[1] = nn.Conv2d(in_channels, num_classes, kernel_size=(1, 1), stride=(1, 1))
        else:
            print('inception / densenet are not available on this environment')
            raise NotImplementedError
        return model
    else:
        return models.__dict__[model_arch](pretrained=False, num_classes=num_classes)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import timedelta
import time
from tqdm import tqdm


class Trainer:
    def __init__(self, model, optimizer, cfg, device, train_loader, val_loader=None):
        self.cfg = cfg
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.metric_logger = MetricLogger(delimeter=" | ")
        self.metric_logger.add_meter('lr', AverageMeter(window_size=1, fmt='{value:.6f}'))
        self.metric_logger.add_meter('loss', AverageMeter(window_size=1, fmt='{value:.6f}'))
        self.metric_logger.add_meter('acc', AverageMeter(window_size=1, fmt='{value:.2f}%'))
        
        print("using gpu {}".format(cfg.SYSTEM.GPU))
        
        self.model = model.to(device)
        
        update_params = [p for p in model.parameters() if p.requires_grad]
        self.optimizer = optimizer(update_params, lr=cfg.TRAIN.BASE_LR)
        self.criterion = nn.CrossEntropyLoss()
        self.total_step = 0
        self.training_time = 0
        
        print(model)
        print()
        print(optimizer)
        print()
               
    def train(self):
        print("training start!\n")
        
        for epoch in range(self.cfg.TRAIN.EPOCH):
            self.train_one_epoch(epoch)
            if epoch % self.cfg.TRAIN.VAL_PERIOD == 0:
                self.validate(epoch)
            
        
        saved_path = save_model(self.model, self.optimizer, epoch, self.total_step, self.cfg, is_final=True)    
        print('saved finael model {}'.format(saved_path))        
        print("total train time {}".format(str(timedelta(seconds=int(self.training_time)))))
        
    def train_one_epoch(self, epoch):
        header = 'EPOCH {}'.format(epoch)        
        self.model.train()
        start = time.time()
        for step, (images, labels) in enumerate(self.metric_logger.log_every(self.train_loader, self.cfg.SYSTEM.PRINT_FREQ, header)):
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            logits = self.model(images)
            loss = self.criterion(logits, labels) 
            acc = logits.argmax(-1).eq(labels).float().mean() * 100.
            self.metric_logger.update(loss=loss.detach().cpu().item(), 
                                      acc=acc.detach().cpu(),
                                      lr=self.optimizer.param_groups[0]['lr'])

            if self.total_step % self.cfg.TRAIN.SAVE_PERIOD == 0 and self.total_step != 0:
                saved_path = save_model(self.model, self.optimizer, epoch, self.total_step, self.cfg)    
                print('saved model {}'.format(saved_path))        

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            self.total_step += 1
        
        end = time.time()
        epoch_train_time = end - start
        self.training_time += epoch_train_time
        print("EPOCH {} train time {}".format(epoch, str(timedelta(seconds=int(epoch_train_time)))))
        
    def validate(self, epoch):
        self.model.eval()
        print('calculating accuracy on val set...')
        corrects = 0.
        num_data = len(self.val_loader.dataset)
        with torch.no_grad():
            for images, labels in tqdm(self.val_loader):
                images = images.to(self.device)
                labels = labels .to(self.device)

                logits = self.model(images)
                corrects += logits.argmax(-1).eq(labels).float().sum()

            val_acc = (corrects.cpu().item() / num_data) * 100.
            print("EPOCH {} VAL ACCURACY : {:.2f}".format(epoch, val_acc))

In [ ]:
import torch
import torch.optim as optim
import os 
from datetime import datetime

def main(config):
    date = datetime.now()
    folder_name = '{}_{}_{}_{}'.format(date.month, date.day, date.hour, date.minute)

    config.SAVE_DIR = os.path.join(config.BASE_SAVE_DIR, folder_name)
  
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    data_root = config.DATASET.ROOT
    
    transform = get_augmentation(**config.TRAIN.AUGMENTATION)
    train_dataset = LandMarkDataset(data_root, config, 'train', transform=transform)
    train_loader = get_loader(train_dataset, config)
    val_dataset = LandMarkDataset(data_root, config, 'val', transform=transform)
    val_loader = get_loader(val_dataset, config)

    model = get_convnet(config.MODEL.ARCH, config.MODEL.USE_PRETRAINED, config.MODEL.NUM_CLASSES, config.TRAIN.USE_ADVPROP)
    optimizer = optim.__dict__[config.MODEL.OPTIM]

    trainer = Trainer(model, optimizer, config, device, train_loader, val_loader)
    trainer.train()

In [ ]:
config = ConfigTree()

config.DATASET.ROOT = "./data" # 데이터 위치
config.DATASET.NUM_CLASSES = 5 # 분류해야 하는 클래스 종류의 수 

config.BASE_SAVE_DIR = './outputs' # 모델 파라미터가 저장되는 위치

config.SYSTEM.GPU = 0 # GPU 번호 
config.SYSTEM.PRINT_FREQ = 5 # 로그를 프린트하는 주기

# 실험 요소
config.TRAIN.EPOCH = 30 # 총 학습 에폭
config.TRAIN.BATCH_SIZE = 28 # 배치 사이즈
config.TRAIN.BASE_LR = 0.01 # 러닝 레이트
config.TRAIN.VAL_PERIOD = 1 # validation accuracy 측정 주기
config.TRAIN.SAVE_PERIOD = 100 # 모델 저장하는 주기
config.TRAIN.USE_ADVPROP = True
config.TRAIN.AUGMENTATION = {'size' : 224, 'use_flip' : True, 'use_gray_scale' : False, 'use_normalize' : True} # 사용할 augmentation

config.MODEL.ARCH = 'efficientnet-b5' # 모델 구조
config.MODEL.USE_PRETRAINED = True # ImaegNet 사전 학습 모델 사용
config.MODEL.OPTIM = 'SGD' # 파리미터 업데이터(optimizer) 
config.MODEL.NUM_CLASSES = 5

config.TEAM_NAME = 'action' # 소속 팀

In [ ]:
if __name__ == '__main__':
    main(config)

Loaded pretrained weights for efficientnet-b5
using gpu 0
EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_

  0%|          | 0/18 [00:00<?, ?it/s]

EPOCH 0 | [89/90] | eta: 0:00:01 | lr: 0.010000 | loss: 1.365735 | acc: 50.00%
EPOCH 0 train time 0:02:42
calculating accuracy on val set...


100%|██████████| 18/18 [00:23<00:00,  1.31s/it]


EPOCH 0 VAL ACCURACY : 76.40
EPOCH 1 | [ 0/90] | eta: 0:02:52 | lr: 0.010000 | loss: 1.295697 | acc: 71.43%
EPOCH 1 | [ 5/90] | eta: 0:02:35 | lr: 0.010000 | loss: 1.263377 | acc: 67.86%
saved model ./outputs/11_28_12_56/efficientnet-b5_action_100.pth
EPOCH 1 | [10/90] | eta: 0:02:30 | lr: 0.010000 | loss: 1.247713 | acc: 67.86%
EPOCH 1 | [15/90] | eta: 0:02:19 | lr: 0.010000 | loss: 1.172621 | acc: 75.00%
EPOCH 1 | [20/90] | eta: 0:02:09 | lr: 0.010000 | loss: 1.249317 | acc: 67.86%
EPOCH 1 | [25/90] | eta: 0:01:59 | lr: 0.010000 | loss: 1.125670 | acc: 85.71%
EPOCH 1 | [30/90] | eta: 0:01:50 | lr: 0.010000 | loss: 1.225309 | acc: 64.29%
EPOCH 1 | [35/90] | eta: 0:01:41 | lr: 0.010000 | loss: 1.194712 | acc: 64.29%
EPOCH 1 | [40/90] | eta: 0:01:31 | lr: 0.010000 | loss: 1.181506 | acc: 67.86%
EPOCH 1 | [45/90] | eta: 0:01:22 | lr: 0.010000 | loss: 1.098019 | acc: 89.29%
EPOCH 1 | [50/90] | eta: 0:01:13 | lr: 0.010000 | loss: 1.079602 | acc: 75.00%
EPOCH 1 | [55/90] | eta: 0:01:04 | lr

...

EPOCH 28 VAL ACCURACY : 94.80
EPOCH 29 | [ 0/90] | eta: 0:02:37 | lr: 0.010000 | loss: 0.239935 | acc: 89.29%
EPOCH 29 | [ 5/90] | eta: 0:02:29 | lr: 0.010000 | loss: 0.117200 | acc: 96.43%
EPOCH 29 | [10/90] | eta: 0:02:20 | lr: 0.010000 | loss: 0.064848 | acc: 96.43%
EPOCH 29 | [15/90] | eta: 0:02:12 | lr: 0.010000 | loss: 0.068166 | acc: 100.00%
EPOCH 29 | [20/90] | eta: 0:02:03 | lr: 0.010000 | loss: 0.261053 | acc: 92.86%
EPOCH 29 | [25/90] | eta: 0:01:54 | lr: 0.010000 | loss: 0.219214 | acc: 89.29%
EPOCH 29 | [30/90] | eta: 0:01:45 | lr: 0.010000 | loss: 0.043992 | acc: 100.00%
EPOCH 29 | [35/90] | eta: 0:01:36 | lr: 0.010000 | loss: 0.276708 | acc: 92.86%
EPOCH 29 | [40/90] | eta: 0:01:28 | lr: 0.010000 | loss: 0.018510 | acc: 100.00%
EPOCH 29 | [45/90] | eta: 0:01:19 | lr: 0.010000 | loss: 0.161740 | acc: 92.86%
EPOCH 29 | [50/90] | eta: 0:01:10 | lr: 0.010000 | loss: 0.082750 | acc: 96.43%
EPOCH 29 | [55/90] | eta: 0:01:01 | lr: 0.010000 | loss: 0.235228 | acc: 92.86%
EPOCH 2

  0%|          | 0/18 [00:00<?, ?it/s]

EPOCH 29 | [89/90] | eta: 0:00:01 | lr: 0.010000 | loss: 0.091267 | acc: 100.00%
EPOCH 29 train time 0:02:36
calculating accuracy on val set...


100%|██████████| 18/18 [00:22<00:00,  1.23s/it]


EPOCH 29 VAL ACCURACY : 96.60
saved finael model ./outputs/11_28_12_56/efficientnet-b5_action_final.pth
total train time 1:20:37
